<a href="https://colab.research.google.com/github/froggyroot/2024bootcamp/blob/main/%5BTUstore_ord%5D%EB%B6%84%EC%84%9D%EB%B0%A9%EB%B2%95%EB%B0%8F%ED%95%B4%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[분석 목적]

: 상품 대분류 및 고객 분류


[가설 설정]

:


> 1. 기술통계

> 2. 상관관계 분석

- 인과관계 반영 X, 선형 관계 반영 O



> 3. 가설 검정


> 4. 회귀 분석


- 단순 회귀 분석

- 로지스틱 회귀 분석


In [ ]:
# ord_weather 열의 종류와 그 개수를 구하기
ord_weather_counts = df['ord_weather'].value_counts()

# 결과 출력
print(ord_weather_counts)

ord_weather
맑음    28578
눈      5378
흐림     4146
비      1898
Name: count, dtype: int64


In [ ]:
# 전처리 [ 이전 전처리에서 - Set_info, Food_class 컬럼 추가 ]

import pandas as pd

# csv 파일 읽어오는 함수 (read_csv)
df = pd.read_csv("/content/tustore_ord.csv", encoding='cp949', sep=';')

# 데이터 잘 구분되어 들어왔는지 확인.
df.head()

# beverage, Set, alcohol, snack을 구분하는 함수 정의
def categorize_goods(goods_nm):
    beverage = ['생수', '코카콜라', '아이스 아메리카노', '아메리카노(HOT)', '토레타', '카페라떼', '핫초코',
          '미닛메이드 오렌지', '아이스티(복숭아)', '미닛메이드 포도', '허브티', '밀키스', '쵸코라떼 (HOT)','스프라이트']
    Set = ['Sweet ComboSet', 'Double ComboSet', 'Single Set', 'Family set', 'Beer set']
    alcohol = ['테라(캔)', '무알콜 맥주', '사케', '하이볼']

    if goods_nm in beverage:
        return 'beverage'
    elif goods_nm in Set:
        return 'Set'
    elif goods_nm in alcohol:
        return 'alcohol'
    else:
        return 'snack'

# 카테고리별로 구분할 'type' 컬럼 추가
df['type'] = df['goods_nm'].apply(categorize_goods)

# 날짜 형식으로 변환
df['ord_dt'] = pd.to_datetime(df['ord_dt'], format='%Y%m%d')

# 시간 형식 변환 ('HH:MM' 형식)
df['ord_time'] = df['ord_time'].astype(str).str.zfill(4)
df['ord_time'] = df['ord_time'].str[:2] + ':' + df['ord_time'].str[2:]

# ord_id, goods_cd, store_nm 열 삭제
df.drop(['ord_id', 'goods_cd', 'store_nm'], axis=1, inplace=True)

# ord_dt와 ord_time 기준으로 오름차순 정렬
df.sort_values(by=['ord_dt', 'ord_time'], ascending=True, inplace=True)

# s_num, d_num 컬럼 추가 및 Set 분할
def sd_add(row):
    if row['type'] == 'Set':
        if row['goods_nm'] == 'Single Set':
            s_num = 1 * row['ord_qty']
            d_num = 1 * row['ord_qty']
        elif row['goods_nm'] in ['Sweet ComboSet', 'Double ComboSet', 'Beer set']:
            s_num = 1 * row['ord_qty']
            d_num = 2 * row['ord_qty']
        elif row['goods_nm'] == 'Family set':
            s_num = 4 * row['ord_qty']
            d_num = 4 * row['ord_qty']

        row['s_num'] = s_num
        row['d_num'] = d_num
    else:
        if row['type'] == 'snack':
            row['s_num'] = row['ord_qty']
            row['d_num'] = 0
        elif row['type'] in ['beverage', 'alcohol']:
            row['s_num'] = 0
            row['d_num'] = row['ord_qty']

    return row

df = df.apply(sd_add, axis=1)

# Set_info 컬럼 추가
def set_info(goods_nm):
    if goods_nm == 'Sweet ComboSet':
        return '팝콘(L)1개+음료2'
    elif goods_nm == 'Single set' or goods_nm == 'Single Set':
        return '핫도그1개+음료1'
    elif goods_nm == 'Beer set':
        return '닭강정1개+맥주2'
    elif goods_nm == 'Family set':
        return '팝콘(M)2개 + 츄러스2+음료2+아메리카노2'
    else:
        return 'not_set'

df['Set_info'] = df['goods_nm'].apply(set_info)

# Food_class 컬럼 추가
def food_class(goods_nm, set_info):
    if goods_nm in ['캬라멜팝콘(M)', '캬라멜 팝콘(L)', '오리지널 팝콘(M)', '오리지널 팝콘(L)']:
        return 'Popcorn'
    elif goods_nm in ['츄러스','프레즐']:
        return 'Bread'
    elif goods_nm in ['치즈핫도그', '후라이드치킨', '핫바']:
        return 'Fried'
    elif goods_nm in ['코카콜라', '스프라이트', '밀키스']:
        return 'Soda'
    elif goods_nm == '아이스 아메리카노':
        return 'ice_coffee'
    elif goods_nm in ['아메리카노(HOT)', '카페라떼']:
        return 'hot_coffee'
    elif goods_nm in ['생수', '아이스티(복숭아)', '토레타', '미닛메이드 오렌지', '미닛메이드 포도']:
        return 'ice_drink'
    elif goods_nm in ['핫초코', '쵸코라떼 (HOT)']:
        return 'hot_drink'
    elif goods_nm == '스트로베리 조각케잌':
        return 'Dessert'
    elif goods_nm == '허브티':
        return 'tea'
    elif goods_nm in ['테라(캔)', '무알콜 맥주', '하이볼', '사케']:
        return 'Alcohol'
    elif goods_nm in ['하겐다즈 쵸코', '하겐다즈 딸기', '하겐다즈 바닐라']:
        return 'ice_cream'
    elif set_info == '팝콘(L)1개+음료2':
        return 'Popcorn, drink'
    elif set_info == '핫도그1개+음료1':
        return 'Snack, drink'
    elif set_info == '닭강정1개+맥주2':
        return 'Snack, Alcohol'
    elif set_info == '팝콘(M)2개 + 츄러스2+음료2+아메리카노2':
        return 'Popcorn, Snack, drink, coffee'
    else:
        return None

df['Food_class'] = df.apply(lambda row: food_class(row['goods_nm'], row['Set_info']), axis=1)

# 필요한 열만 선택
df = df[['ord_dt', 'goods_nm', 'store_id', 'billno', 'ord_time', 'ord_qty', 'ord_amt', 'ord_day', 'ord_weather', 'type', 's_num', 'd_num', 'Set_info', 'Food_class']]

# 1_tustore_ord.csv 파일로 저장
df.to_csv('1_tustore_ord.csv', index=False, encoding='cp949', sep=';')

df.head(20)

,ord_dt,goods_nm,store_id,billno,ord_time,ord_qty,ord_amt,ord_day,ord_weather,type,s_num,d_num,Set_info,Food_class
16588,2023-10-19,아이스티(복숭아),3,1,19:01,1,4545,수,맑음,beverage,0,1,not_set,ice_drink
25066,2023-10-19,생수,3,1,19:01,1,909,수,맑음,beverage,0,1,not_set,ice_drink
14856,2023-10-19,스프라이트,1,1,19:03,1,3182,수,맑음,beverage,0,1,not_set,Soda
2355,2023-10-19,생수,5,1,19:04,1,909,수,맑음,beverage,0,1,not_set,ice_drink
18389,2023-10-19,프레즐,5,1,19:04,1,4545,수,맑음,snack,1,0,not_set,Bread
30070,2023-10-19,후라이드치킨,2,1,19:04,1,7273,수,맑음,snack,1,0,not_set,Fried
30992,2023-10-19,후라이드치킨,5,1,19:04,1,7273,수,맑음,snack,1,0,not_set,Fried
36441,2023-10-19,아이스 아메리카노,5,1,19:04,1,4545,수,맑음,beverage,0,1,not_set,ice_coffee
25006,2023-10-19,생수,5,3,19:05,1,909,수,맑음,beverage,0,1,not_set,ice_drink
12874,2023-10-19,츄러스,3,2,19:06,1,4545,수,맑음,snack,1,0,not_set,Bread


In [ ]:
import pandas as pd

# csv 파일 읽어오기
df = pd.read_csv("/content/tustore_ord.csv", encoding='cp949', sep=';')

# 데이터 타입 확인
print(df.dtypes)

ord_id          int64
ord_dt          int64
goods_nm       object
store_id        int64
billno          int64
ord_time        int64
ord_qty         int64
ord_amt         int64
goods_cd        int64
ord_day        object
ord_weather    object
store_nm       object
dtype: object


In [ ]:
df = pd.read_csv("1_tustore_ord.csv", encoding='cp949', sep=';')

print(df.dtypes)

ord_dt         object
goods_nm       object
store_id        int64
billno          int64
ord_time       object
ord_qty         int64
ord_amt         int64
ord_day        object
ord_weather    object
type           object
s_num           int64
d_num           int64
Set_info       object
Food_class     object
dtype: object


# 가설 설정

[ 가설 설정을 Type, Set_info(새로 만든 컬럼) : Set 메뉴인 거 위주로 가설 설정했습니다. ]


- 1번 가설

  : type이 Set인 판매량이 ord_day가 "월","화","수","목"일 때보다  "금","토","일" 일 때 더 많을 것이다.



- 2번 가설  

  : type이 alcohol이거나, Set_info가 "닭강정1개+ 맥주2"인 판매량은 전체 판매량의 20%가 되지 않을 것이다.


In [ ]:
# 1번 가설 [: type이 Set인 판매량이 ord_day가 "월","화","수","목"일 때보다 "금","토","일" 일 때 더 많을 것이다.]

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import logit

# csv 파일 읽어오기
df = pd.read_csv("1_tustore_ord.csv", encoding='cp949', sep=';')

# '평일'과 '주말'로 나누기
weekday_set = df[(df['ord_day'].isin(['월', '화', '수', '목'])) & (df['type'] == 'Set')]
weekend_set = df[(df['ord_day'].isin(['금', '토', '일'])) & (df['type'] == 'Set')]

# 기술통계
weekday_set_count = weekday_set['ord_qty'].sum()
weekend_set_count = weekend_set['ord_qty'].sum()

print(f"평일에 'Set' 판매량: {weekday_set_count}")
print(f"주말에 'Set' 판매량: {weekend_set_count}")

# t-검정
t_stat, p_val = ttest_ind(weekday_set['ord_qty'], weekend_set['ord_qty'], equal_var=False)
print(f"t-검정 통계량: {t_stat}, p-값: {p_val}")

# 로지스틱 회귀 분석
df['weekend'] = df['ord_day'].apply(lambda x: 1 if x in ['금', '토', '일'] else 0)
df['set_order'] = df['type'].apply(lambda x: 1 if x == 'Set' else 0)

logit_model = logit("set_order ~ weekend", data=df).fit()
print(logit_model.summary())


평일에 'Set' 판매량: 501
주말에 'Set' 판매량: 1629
t-검정 통계량: 0.9025424021366956, p-값: 0.3671363357346089
Optimization terminated successfully.
         Current function value: 0.197554
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:              set_order   No. Observations:                40000
Model:                          Logit   Df Residuals:                    39998
Method:                           MLE   Df Model:                            1
Date:                Fri, 12 Jul 2024   Pseudo R-squ.:               1.510e-05
Time:                        06:45:35   Log-Likelihood:                -7902.2
converged:                       True   LL-Null:                       -7902.3
Covariance Type:            nonrobust   LLR p-value:                    0.6252
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -

In [ ]:
# 2번 가설 [type이 alcohol이거나, Set_info가 "닭강정1개+ 맥주2"인 판매량은 전체 판매량의 20%가 되지 않을 것이다.]

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import logit

# csv 파일 읽어오기
df = pd.read_csv("1_tustore_ord.csv", encoding='cp949', sep=';')

# 필요한 열만 선택
df = df[['ord_dt', 'goods_nm', 'type', 'ord_qty', 'Set_info']]

# Set_info가 "닭강정1개+맥주2"인 경우와 type이 alcohol인 경우 필터링
df_filtered = df[(df['Set_info'] == '닭강정1개+맥주2') | (df['type'] == 'alcohol')]

# 전체 판매량 계산
total_sales = df['ord_qty'].sum()

# alcohol이거나 Set_info가 "닭강정1개+맥주2"인 판매량 계산
filtered_sales = df_filtered['ord_qty'].sum()

# 전체 판매량 대비 비율 계산
ratio = filtered_sales / total_sales

# 독립변수 추가 (1: alcohol 또는 "닭강정1개+맥주2", 0: 그 외)
df_filtered['is_target'] = np.where((df_filtered['Set_info'] == '닭강정1개+맥주2') | (df_filtered['type'] == 'alcohol'), 1, 0)

# 단순 회귀 분석을 위한 모델 설정
X = df_filtered['is_target']
y = df_filtered['ord_qty']

# 상수항 추가
X = sm.add_constant(X)

# OLS 모델 적합
model = sm.OLS(y, X).fit()

# 회귀 분석 결과 요약
print(model.summary())

# 전체 판매량의 20%와 비교하여 가설 검정
print(f"전체 판매량 대비 alcohol 또는 '닭강정1개+맥주2'인 판매량 비율: {ratio:.2%}")

                            OLS Regression Results                            
Dep. Variable:                ord_qty   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 12 Jul 2024   Prob (F-statistic):                nan
Time:                        06:45:58   Log-Likelihood:                -2717.0
No. Observations:                2018   AIC:                             5436.
Df Residuals:                    2017   BIC:                             5442.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
is_target      1.3979      0.021     67.505      0.0

<ipython-input-22-f2039d2a8851>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['is_target'] = np.where((df_filtered['Set_info'] == '닭강정1개+맥주2') | (df_filtered['type'] == 'alcohol'), 1, 0)
